<a href="https://colab.research.google.com/github/chakrateja70/claude-Agent/blob/main/ClaudeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://chatgpt.com/c/67a198f6-fd28-800c-9a08-0e0dd41e3ac4

# *Required Pacakges*

In [ ]:
!pip install langchain langchain_anthropic pinecone pinecone-client anthropic voyageai langchain-voyageai python-dotenv pypdf langchain-community datasets


In [ ]:
from google.colab import userdata
userdata.get('PINECONE_API_KEY')
userdata.get('ANTHROPIC_API_KEY')
userdata.get('VOYAGE_API_KEY')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    pages = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        pages.extend(loader.load())
    return pages

folder_path = "/content/"
pages = load_documents(folder_path)
print(f"Loaded {len(pages)} documents from the folder.")


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.split_documents(pages)
print(f"split the document into  {len(splits)} chunks.")

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

document_embeddings

In [ ]:
!pip install langchain-pinecone


If you get error regarding index, create index manually ( dimensions= 384)


In [ ]:

from langchain.vectorstores import Pinecone

import os

index_name = "quickstart"

pinecone_api_key = userdata.get('PINECONE_API_KEY')

# pc.create_index(
#     name="agent",
#     dimension=384, # Replace with your model dimensions
#     metric="cosine", # Replace with your model metric
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain.vectorstores import Pinecone
vectordb = Pinecone.from_documents(
    documents=splits,
    embedding=embedding_function,
    index_name=index_name,
)

print(f"Data successfully stored in Pinecone under index '{index_name}'!")

In [ ]:
import os
from langchain_anthropic import ChatAnthropic
from langchain.chains import RetrievalQA
from getpass import getpass

anthropic_api_key = userdata.get('ANTHROPIC_API_KEY') or getpass("Anthropic API key: ")

# chat completion llm
llm = ChatAnthropic(
    anthropic_api_key=anthropic_api_key,
    model_name="claude-3-sonnet-20240229",  # change "opus" -> "sonnet" for speed
    temperature=0.0
)

retriever = vectordb.as_retriever()
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("RAG pipeline successfully set up!")

In [ ]:
# User Query
query = "what is FY26?"

# Get Response from RAG System
response = rag_chain.invoke({"query": query})

# Check if response is empty
if not response.get("result"):
    print("No relevant information found in the knowledge base.")
else:
    print("Answer:", response["result"])
